In [2]:
import pandas as pd
import numpy as np
import math
import re

In [3]:
# t table
dict1={0.5:[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],0.25:[1.000,0.816,0.765,0.741,0.727,0.718,0.711,0.706,0.703,0.700,0.697,0.695,0.694,0.692,0.691,0.690,0.689,0.688,0.688,0.687,0.686,0.686,0.685,0.685,0.684,0.684,0.684,0.683,0.683,0.683],0.2:[1.376,1.061,0.978,0.941,0.920,0.906,0.896,0.889,0.883,0.879,0.876,0.873,0.870,0.868,0.866,0.865,0.863,0.862,0.861,0.860,0.859,0.858,0.858,0.857,0.856,0.856,0.855,0.855,0.854,0.854],0.15:[1.963,1.386,1.250,1.190,1.156,1.134,1.119,1.108,1.100,1.093,1.088,1.083,1.079,1.076,1.074,1.071,1.069,1.067,1.066,1.064,1.063,1.061,1.060,1.059,1.058,1.058,1.057,1.056,1.055,1.055],0.10:[3.078,1.886,1.638,1.533,1.476,1.440,1.415,1.397,1.383,1.372,1.363,1.356,1.350,1.345,1.341,1.337,1.333,1.330,1.328,1.325,1.323,1.321,1.319,1.318,1.316,1.315,1.314,1.313,1.311,1.310],0.05:[6.314,2.920,2.353,2.132,2.015,1.943,1.895,1.860,1.833,1.812,1.796,1.782,1.771,1.761,1.753,1.746,1.740,1.734,1.729,1.725,1.721,1.717,1.714,1.711,1.708,1.706,1.703,1.701,1.699,1.697], 0.025:[12.71 ,4.303 ,3.182 ,2.776 ,2.571 ,2.447 ,2.365 ,2.306 ,2.262 ,2.228,2.201,2.179,2.160,2.145,2.131,2.120,2.110,2.101,2.093,2.086,2.080,2.074,2.069,2.064,2.060,2.056,2.052,2.048,2.045,2.042],0.01:[31.82,6.965,4.541,3.747,3.365,3.143,2.998,2.896,2.821,2.764,2.718,2.681,2.650,2.624,2.602,2.583,2.567,2.552,2.539,2.528,2.518,2.508,2.500,2.492,2.485,2.479,2.473,2.467,2.462,2.457],0.005:[63.66,9.925,5.841,4.604,4.032,3.707,3.499,3.355,3.250,3.169,3.106,3.055,3.012,2.977,2.947,2.921,2.898,2.878,2.861,2.845,2.831,2.819,2.807,2.797,2.787,2.779,2.771,2.763,2.756,2.750],0.001:[318.31,22.327,10.215,7.173 ,5.893 ,5.208 ,4.785 ,4.501 ,4.297 ,4.144 ,4.025 ,3.930 ,3.852 ,3.787 ,3.733 ,3.686 ,3.646 ,3.610 ,3.579 ,3.552 ,3.527 ,3.505 ,3.485 ,3.467 ,3.450 ,3.435 ,3.421 ,3.408 ,3.396 ,3.385 ],0.0005:[636.62,31.599,12.924,8.610,6.869,5.959,5.408,5.041,4.781,4.587,4.437,4.318,4.221,4.140,4.073,4.015,3.965,3.922,3.883,3.850,3.819,3.792,3.768,3.745,3.725,3.707,3.690,3.674,3.659,3.646]}
df_t=pd.DataFrame(dict1, index=list(range(1,31)))

In [4]:
def hypothesis_and_estimate():
    '''
    Finds point estimate, interval estimate and does hypothesis testing
    
    Inputs :
    hyp : dict, default {'Ho':'mu>=8','Ha':'mu<8'}
          It accepts hypothesis statement as a dictionary
        
    alpha: float, default 0.1 (press Enter to select the default value)
           Significance level. alpha=1-confidence interval.
           Values - 0.1, 0.05 (for sample size>30 - z statistic)
           1.0, 0.5, 0.4, 0.3, 0.2, 0.1, 0.05, 0.02, 0.01, 0.002, 0.001 (for sample size<30 - t statistic)
    
    
    '''
    
    z={0.1:1.28, 0.05:1.645, 0.025:1.96}
    flag=0
    
    ## ---------------------------user input--------------------------------
 
    inp2 = input('''
    Enter one of the following 
    1 : Point Estimate
    2 : Interval Estimate
    3 : Hypothesis testing
    4 : All
    any other key : quit
    
    ''')
    if inp2 in ('1','2','4'):
        inp3 = input('''
        How do you want to provide the data?
        1 : CSV
        2 : Direct sample data 
        any other key : quit
        
        ''')
        if inp3=='1':
            csv = input('Enter csv file name: ')
            col = int(input('Column number in the csv to analyze (default 0): ') or 0)
            
            try:
                df=pd.read_csv(csv)
            except:
                print("Enter valid csv file name")
                return
            else:
                # Selecting the required data from the csv
                df1=df.select_dtypes(include=np.number) # remove non numeric columns
                col_name=df1.columns.values[col]  #column name
                print(col_name," data")
                df2=df1.iloc[:,col]   #get the column values            
                
                sample_size=df2.count()
                sample_mean=df2.mean()
                sample_std=df2.std()
            
        elif inp3=='2' and inp2!='1':    # for point estimate only, we dont need everything
            sample_mean, sample_std, sample_size = eval(input("Sample mean,Sample std,sample size: "))

        elif inp3=='2' and inp2=='1':  # for point estimate 
            sample_mean=float(input('Sample mean: '))
        else:
            return
        
    elif inp2=='3':  
        sample_mean, sample_size = eval(input("Sample mean, Sample size: "))

    else:
        return
    
    # asking for population std
    if inp2 =='2':
        pop_std = eval(input('Population std(if available or else press enter): ') or 'None')
    elif inp2 in ('3','4'):
        pop_std = float(input('Population std: '))
       
    if inp2!='1':  #asking for alpha (not when we just want point estimate)
        alpha = float(input("Enter alpha value (press Enter to select the default value - 0.1): ") or 0.1)
    
    ## -----------------------Point estimate-------------------------------------
    
    if inp2 in ('1','4'):
        print('\nPoint estimate = ', sample_mean,'\n')
        
    ## -----------------------Hypothesis testing----------------------------------
    
    if inp2 in ('3','4'):
        hyp=eval(input("Enter hypothesis (default - {'Ho':'mu>=8','Ha':'mu<8'} press Enter to select): ") or "{'Ho':'mu>=8','Ha':'mu<8'}")
        mu=int(re.findall(r'\d+',hyp['Ho'])[0])
        z_stats=(sample_mean-mu)/(pop_std/math.sqrt(sample_size))
        
        # comparison of z-alpha and z-stats based on the tail(s)
        try:
            if hyp['Ho'].find('>')!=-1:
                print('\nThis is a z-distribution with lower tail')
                if -z[alpha]>=z_stats:
                    print('Null hypothesis rejected based on the sample provided')
                else:
                    print('Null hypothesis accepted based on the sample provided')
            elif hyp['Ho'].find('<')!=-1:
                print('\nThis is a z-distribution with upper tail')
                if z[alpha]<=z_stats:
                    print('Null hypothesis rejected based on the sample provided')
                else:
                    print('Null hypothesis accepted based on the sample provided')
            else:
                print('\nThis is a z-distribution with two tails')
                if z[alpha/2]<=abs(z_stats):   # abs is used if z_stats comes out to be in the lower extreme (when sample_mean<mu)
                    print('Null hypothesis rejected based on the sample provided')
                else:
                    print('Null hypothesis accepted based on the sample provided')
        except:
            print('Check the documentation for the available alpha value')
    
       
    ## -------------------------------Calculations for IE----------------------------------
    
    if inp2 in ('2','4'):
        try:
            if sample_size>30:
                flag=1
                std=pop_std if pop_std!=None else sample_std  # if population sd is provided use that for the z statistic
                E=z[alpha/2]*std/math.sqrt(sample_size) 
            else:
                std=sample_std
                E=df_t[alpha/2][sample_size-1]*std/math.sqrt(sample_size)
        except:
            print("Check the documentation for the available alpha value")
        else:
            interval_estimate = str(sample_mean-E)+ ' to ' + str(sample_mean+E)
            txt="z" if flag==1 else "t"
            print(f'\nThis is a {txt} distribution with')
            print('Interval estimate = ', interval_estimate)
        

In [48]:
hypothesis_and_estimate()


    Enter one of the following 
    1 : Point Estimate
    2 : Interval Estimate
    3 : Hypothesis testing
    4 : All
    any other key : quit
    
    1

        How do you want to provide the data?
        1 : CSV
        2 : Direct sample data 
        any other key : quit
        
        2
Sample mean: 23

Point estimate =  23.0 



In [49]:
hypothesis_and_estimate()


    Enter one of the following 
    1 : Point Estimate
    2 : Interval Estimate
    3 : Hypothesis testing
    4 : All
    any other key : quit
    
    1

        How do you want to provide the data?
        1 : CSV
        2 : Direct sample data 
        any other key : quit
        
        1
Enter csv file name: sample(z-stat).csv
Column number in the csv to analyze (default 0): 
Salary  data

Point estimate =  180921.19589041095 



In [50]:
hypothesis_and_estimate()


    Enter one of the following 
    1 : Point Estimate
    2 : Interval Estimate
    3 : Hypothesis testing
    4 : All
    any other key : quit
    
    2

        How do you want to provide the data?
        1 : CSV
        2 : Direct sample data 
        any other key : quit
        
        2
Sample mean,Sample std,sample size: 45,3.2,23
Population std(if available or else press enter): 
Enter alpha value (press Enter to select the default value - 0.1): 

This is a t distribution with
Interval estimate =  43.85433840340481 to 46.14566159659519


In [51]:
hypothesis_and_estimate()


    Enter one of the following 
    1 : Point Estimate
    2 : Interval Estimate
    3 : Hypothesis testing
    4 : All
    any other key : quit
    
    3
Sample mean, Sample size: 8.2,34
Population std: 0.5
Enter alpha value (press Enter to select the default value - 0.1): 0.05
Enter hypothesis (default - {'Ho':'mu>=8','Ha':'mu<8'} press Enter to select): 

This is a z-distribution with lower tail
Null hypothesis accepted based on the sample provided


In [53]:
hypothesis_and_estimate()


    Enter one of the following 
    1 : Point Estimate
    2 : Interval Estimate
    3 : Hypothesis testing
    4 : All
    any other key : quit
    
    4

        How do you want to provide the data?
        1 : CSV
        2 : Direct sample data 
        any other key : quit
        
        1
Enter csv file name: sample(z-stat).csv
Column number in the csv to analyze (default 0): 
Salary  data
Population std: 2.5
Enter alpha value (press Enter to select the default value - 0.1): 

Point estimate =  180921.19589041095 

Enter hypothesis (default - {'Ho':'mu>=8','Ha':'mu<8'} press Enter to select): {'Ho':'mu<=180000','Ha':'mu>180000'}

This is a z-distribution with upper tail
Null hypothesis rejected based on the sample provided

This is a z distribution with
Interval estimate =  180921.08826136688 to 180921.30351945502


In [54]:
hypothesis_and_estimate()


    Enter one of the following 
    1 : Point Estimate
    2 : Interval Estimate
    3 : Hypothesis testing
    4 : All
    any other key : quit
    
    4

        How do you want to provide the data?
        1 : CSV
        2 : Direct sample data 
        any other key : quit
        
        2
Sample mean,Sample std,sample size: 6.1,0.4,30
Population std: 0.2
Enter alpha value (press Enter to select the default value - 0.1): 0.05

Point estimate =  6.1 

Enter hypothesis (default - {'Ho':'mu>=8','Ha':'mu<8'} press Enter to select): {'Ho':'mu=6','Ha':'mu!=6'}

This is a z-distribution with two tails
Null hypothesis rejected based on the sample provided

This is a t distribution with
Interval estimate =  5.950654315986925 to 6.2493456840130746


In [56]:
hypothesis_and_estimate()


    Enter one of the following 
    1 : Point Estimate
    2 : Interval Estimate
    3 : Hypothesis testing
    4 : All
    any other key : quit
    
    3
Sample mean, Sample size: 9,34
Population std: 0.5
Enter alpha value (press Enter to select the default value - 0.1): 0.34
Enter hypothesis (default - {'Ho':'mu>=8','Ha':'mu<8'} press Enter to select): 

This is a z-distribution with lower tail
Check the documentation for the available alpha value


In [59]:
hypothesis_and_estimate()


    Enter one of the following 
    1 : Point Estimate
    2 : Interval Estimate
    3 : Hypothesis testing
    4 : All
    any other key : quit
    
    4

        How do you want to provide the data?
        1 : CSV
        2 : Direct sample data 
        any other key : quit
        
        1
Enter csv file name: sample(t-stat).csv
Column number in the csv to analyze (default 0): 
Salary  data
Population std: 5
Enter alpha value (press Enter to select the default value - 0.1): 

Point estimate =  190435.29411764705 

Enter hypothesis (default - {'Ho':'mu>=8','Ha':'mu<8'} press Enter to select): {'Ho':'mu<=200000','Ha':'mu>200000'}

This is a z-distribution with upper tail
Null hypothesis accepted based on the sample provided

This is a t distribution with
Interval estimate =  161178.29201318906 to 219692.29622210504


In [5]:
hypothesis_and_estimate()


    Enter one of the following 
    1 : Point Estimate
    2 : Interval Estimate
    3 : Hypothesis testing
    4 : All
    any other key : quit
    
    4

        How do you want to provide the data?
        1 : CSV
        2 : Direct sample data 
        any other key : quit
        
        1
Enter csv file name: sample(z-stat).csv
Column number in the csv to analyze (default 0): 
Salary  data
Population std: 5
Enter alpha value (press Enter to select the default value - 0.1): 

Point estimate =  180921.19589041095 

Enter hypothesis (default - {'Ho':'mu>=8','Ha':'mu<8'} press Enter to select): {'Ho':'mu=180000','Ha':'mu!=180000'}

This is a z-distribution with two tails
Null hypothesis rejected based on the sample provided

This is a z distribution with
Interval estimate =  180920.98063232278 to 180921.41114849912
